In [1]:
#import required libraries
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

2023-01-05 15:31:11.394867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 15:31:11.611514: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-05 15:31:11.611544: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-05 15:31:13.239561: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
#using image data generator to import the images from the dataset
#divide dataset into train and validation dataset
train_gen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,
                             horizontal_flip=True,validation_split=0.25)

In [3]:
#train dataset,we have to make sure that same target size as initialized for the image size
train_data=train_gen.flow_from_directory("/home/ubuntu/Downloads/malaria dataset/cell_images/cell_images",
                                          target_size=(224,224),batch_size=64,
                                         class_mode="binary",subset="training")

Found 20670 images belonging to 2 classes.


In [4]:
#validation dataset
validation_data=train_gen.flow_from_directory("/home/ubuntu/Downloads/malaria dataset/cell_images/cell_images",
                                          target_size=(224,224),batch_size=64,
                                              class_mode="binary",subset="validation")

Found 6888 images belonging to 2 classes.


In [5]:
validation_data.class_indices

{'Parasitized': 0, 'Uninfected': 1}

In [13]:
validation_data.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [6]:
mob_net=tensorflow.keras.applications.MobileNetV2(weights="imagenet",input_shape=(224,224,3),include_top=False)

mob_net.trainable=False

2023-01-05 15:32:11.881421: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-05 15:32:11.881455: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-05 15:32:11.881478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-Zako): /proc/driver/nvidia/version does not exist
2023-01-05 15:32:11.883680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
#built architecture
model1=Sequential()

model1.add(mob_net)

model1.add(Flatten())

model1.add(Dense(units=32,activation="relu"))
model1.add(Dropout(0.20))
model1.add(Dense(units=16,activation="relu"))
model1.add(BatchNormalization())
model1.add(Dropout(0.20))
model1.add(Dense(units=1,activation="sigmoid"))

In [8]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 32)                2007072   
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 batch_normalization (BatchN  (None, 16)               64        
 ormalization)                                          

In [9]:
#compile the model
model1.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [10]:
# to avoid overfittig 
early_stop = EarlyStopping(monitor ='val_loss',patience =2)

In [11]:
#fit the model
model1.fit(train_data,validation_data=validation_data,callbacks=[early_stop],epochs=5)

Epoch 1/5


2023-01-05 15:33:36.535074: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2023-01-05 15:33:36.709898: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2023-01-05 15:33:36.823183: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2023-01-05 15:33:36.940057: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 308281344 exceeds 10% of free system memory.
2023-01-05 15:33:37.146190: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 313810944 exceeds 10% of free system memory.


323/323 [==============================] - 884s 3s/step - loss: 0.2771 - accuracy: 0.8874 - val_loss: 0.1815 - val_accuracy: 0.9345
Epoch 2/5
323/323 [==============================] - 841s 3s/step - loss: 0.1909 - accuracy: 0.9315 - val_loss: 0.1777 - val_accuracy: 0.9360
Epoch 3/5
323/323 [==============================] - 834s 3s/step - loss: 0.1800 - accuracy: 0.9351 - val_loss: 0.1601 - val_accuracy: 0.9421
Epoch 4/5
323/323 [==============================] - 852s 3s/step - loss: 0.1716 - accuracy: 0.9382 - val_loss: 0.1820 - val_accuracy: 0.9339
Epoch 5/5
323/323 [==============================] - 841s 3s/step - loss: 0.1681 - accuracy: 0.9388 - val_loss: 0.1607 - val_accuracy: 0.9399


In [12]:
model1.save('malaria_mobnet_model.h5')